In [388]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [389]:
#Please add the folder name of the dataset to run it on different dataset.
dataset = 'AWA2'

In [390]:
#Replace Path to run on your own system
res101 = scipy.io.loadmat('C:/Users/Zach/Documents/NorthEastern/SmallDataMachineLearning/Code/ESZSL/Data/xlsa17/data/'+dataset+'/res101.mat')
att_splits = scipy.io.loadmat('C:/Users/Zach/Documents/NorthEastern/SmallDataMachineLearning/Code/ESZSL/Data/xlsa17/data/'+dataset+'/att_splits.mat')

In [391]:
res101.keys()

dict_keys(['__header__', '__version__', '__globals__', 'features', 'image_files', 'labels'])

In [392]:
att_splits.keys()

dict_keys(['__header__', '__version__', '__globals__', 'allclasses_names', 'att', 'original_att', 'test_seen_loc', 'test_unseen_loc', 'train_loc', 'trainval_loc', 'val_loc'])

In [393]:
#Using the correct naming conventions to get the loctions
trainval_loc = 'trainval_loc'
train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'

We need the corresponding ground-truth labels/classes for each training example for all our train, val, trainval and test set according to the split locations provided.
In this example we have used the `CUB` dataset which has 200 unique classes overall.

In [394]:
#RES101 is Resnet 101, this is the output from CNN
labels = res101['labels']
#Array of training labels (class label)
labels_train = labels[np.squeeze(att_splits[train_loc]-1)]
#array of validation labels
labels_val = labels[np.squeeze(att_splits[val_loc]-1)]
#train + validation labels
labels_trainval = labels[np.squeeze(att_splits[trainval_loc]-1)]
#test labels
labels_test = labels[np.squeeze(att_splits[test_loc]-1)]



In [395]:
print(labels_test[:10])

[[30]
 [30]
 [30]
 [30]
 [30]
 [30]
 [30]
 [30]
 [30]
 [30]]


In [396]:
#Checking first ten labels
labels_train[:10,:]

array([[43],
       [22],
       [43],
       [38],
       [19],
       [18],
       [46],
       [19],
       [49],
       [37]], dtype=uint8)

In [397]:
#list all unique labels, should be 200 for birds
unique_labels = np.unique(labels)
unique_labels

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
      dtype=uint8)

In a typical zero-shot learning scenario, there are no overlapping classes between training and testing phase, i.e the train classes are completely different from the test classes. So let us verify if there are any overlapping classes in the test and train scenario.
- During training phase we have `z` classes
- During the testing phase we have `z'` classes

In [398]:
train_labels_seen = np.unique(labels_train)
val_labels_unseen = np.unique(labels_val)
trainval_labels_seen = np.unique(labels_trainval)
test_labels_unseen = np.unique(labels_test)

In [399]:
print("Number of overlapping classes between train and val:",len(set(train_labels_seen).intersection(set(val_labels_unseen))))
print("Number of overlapping classes between trainval and test:",len(set(trainval_labels_seen).intersection(set(test_labels_unseen))))

Number of overlapping classes between train and val: 0
Number of overlapping classes between trainval and test: 0


In [400]:
#label_train now should be an index into the train_labels seen
#example: anything that had 200 as a label, now is 99 which is the last index in train_labels seen
#which aligns with label 200
i = 0
for labels in train_labels_seen:
    labels_train[labels_train == labels] = i    
    i = i+1
j = 0
for labels in val_labels_unseen:
    labels_val[labels_val == labels] = j
    j = j+1
k = 0
for labels in trainval_labels_seen:
    labels_trainval[labels_trainval == labels] = k
    k = k+1
l = 0
for labels in test_labels_unseen:
    labels_test[labels_test == labels] = l
    l = l+1





Let us denote the features X ∈ [d×m] available at training stage, where d is the dimensionality
of the data, and m is the number of instances. We are useing resnet features which are extracted from `CUB` dataset.

In [401]:
#Get all features of all data
X_features = res101['features']
#select those for the training
train_vec = X_features[:,np.squeeze(att_splits[train_loc]-1)]
#validation set
val_vec = X_features[:,np.squeeze(att_splits[val_loc]-1)]
#train+validation
trainval_vec = X_features[:,np.squeeze(att_splits[trainval_loc]-1)]
#last the test features
test_vec = X_features[:,np.squeeze(att_splits[test_loc]-1)]

In [402]:
#looks like resnet101 outputs 2048 features
print("Features for train:", train_vec.shape)
print("Features for val:", val_vec.shape)
print("Features for trainval:", trainval_vec.shape)
print("Features for test:", test_vec.shape)

Features for train: (2048, 16187)
Features for val: (2048, 7340)
Features for trainval: (2048, 23527)
Features for test: (2048, 7913)


#### Normalize the vectors

In [403]:
def normalization(vec,mean,std):
    sol = vec - mean
    sol1 = sol/std
    return sol1

Each of the classes in the dataset have an attribute (a) description. This vector is known as the `Signature matrix` of dimension S ∈ [0, 1]a×z. For training stage there are z classes and z' classes  for test S ∈ [0, 1]a×z'.

In [404]:
#Attribute Signature matrix (all)
#All normalized to have unit 12 norm
signature = att_splits['att']
#getting the training,validation,train+val,and the test signatures
train_sig = signature[:,(train_labels_seen)-1]
val_sig = signature[:,(val_labels_unseen)-1]
trainval_sig = signature[:,(trainval_labels_seen)-1]
test_sig = signature[:,(test_labels_unseen)-1]

This is a signature matrix, where the occurance of an attribute corresponding to the class is give.
For instance, if the classes are `horse` and `zebra` and the corresponding attributes are [wild_animal, 4_legged, carnivore]

```
 Horse      Zebra
[0.00354613 0.        ] Domestic_animal
[0.13829921 0.20209503] 4_legged
[0.06560347 0.04155225] carnivore
```

In [405]:
#for the Caltech Birds(CUB) there are 311 attributes
print(trainval_sig[3:6,:2])

[[-0.00375358  0.22753174]
 [ 0.0463192   0.01150855]
 [ 0.          0.        ]]


In [406]:
print("Signature for train:", train_sig.shape)
print("Signature for val:", val_sig.shape)
print("Signature for trainval:", trainval_sig.shape)
print("Signature for test:", test_sig.shape)

Signature for train: (85, 27)
Signature for val: (85, 13)
Signature for trainval: (85, 40)
Signature for test: (85, 10)


In [407]:
#params for train and val set
m_train = labels_train.shape[0]
n_val = labels_val.shape[0]
z_train = len(train_labels_seen)
z1_val = len(val_labels_unseen)

#params for trainval and test set
m_trainval = labels_trainval.shape[0]
n_test = labels_test.shape[0]
z_trainval = len(trainval_labels_seen)
z1_test = len(test_labels_unseen)

The ground truth is a one-hot encoded vector

In [408]:
#Setting ground truths for the the classes
#ground truth for train 
gt_train = 0*np.ones((m_train, z_train))
gt_train[np.arange(m_train), np.squeeze(labels_train)] = 1

#grountruth for trainval 
gt_trainval = 0*np.ones((m_trainval, z_trainval))
gt_trainval[np.arange(m_trainval), np.squeeze(labels_trainval)] = 1

In [409]:
gt_train[:1,:100]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [410]:
#train set dimensions
d_train = train_vec.shape[0] #num of dimensions of features
a_train = train_sig.shape[0] #num of attributes

#Weights matrix
#V = np.random.rand(d_train,a_train)
#print(V)

[[0.07514072 0.22781304 0.09419869 ... 0.14812937 0.89535171 0.26911923]
 [0.16373933 0.1186293  0.22807321 ... 0.47775361 0.89335    0.5642668 ]
 [0.44017351 0.38097548 0.07756222 ... 0.32541005 0.32501173 0.93339768]
 ...
 [0.115406   0.200567   0.87017679 ... 0.26915815 0.8351356  0.21022228]
 [0.00708325 0.71062486 0.98077692 ... 0.01685801 0.67188563 0.87969359]
 [0.53139052 0.92698919 0.6167304  ... 0.0345136  0.04086571 0.59086512]]


Need to add the formula in here

In [387]:
import torch
import torch.nn as nn
from sklearn.preprocessing import normalize 

V = torch.tensor(np.random.rand(d_train,a_train) * 1e-5, requires_grad=True)
#print(V)

'''
part_1 = np.linalg.pinv(np.matmul(train_vec, train_vec.transpose()) + (10**1)*np.eye(d_train))
part_0 = np.matmul(np.matmul(train_vec,gt_train),train_sig.transpose())
part_2 = np.linalg.pinv(np.matmul(train_sig, train_sig.transpose()) + (10**1)*np.eye(a_train))

V  = torch.tensor(np.matmul(np.matmul(part_1,part_0),part_2))
'''

#need my values X,V,S, and Y
#V = torch.tensor(V,requires_grad=True)
#going to reassign
X = torch.tensor(train_vec)
S = torch.tensor(train_sig)
Y = torch.tensor(gt_train)
#print(S)
print(Y)

def forward(X,V,S):
    xT = torch.transpose(X,0,1)
    mult1 = torch.matmul(xT,V)
    ans = torch.matmul(mult1,S)
    #print(ans)
    return ans

#regularization term
def reg (V,X,S,g,l,b):

    #Calculate frobenius norm of VS
    part1 = g*(torch.norm(torch.matmul(V,S)))
    #Calculate the Frobenius norm of XTV
    part2 = l * (torch.norm(torch.matmul(torch.transpose(X,0,1),V)))
    #Calculate the frobenius norm of V
    part3 = b * (torch.norm(V))
    return part1 + part2 + part3


def loss_func(y_pred,Y):
    return torch.norm(torch.subtract(y_pred,Y))


#Test values, just getting it to work
iter = 100
learning_rate = 0.01



for i in range(1000):

    if V.grad is not None:
        V.grad.data.zero_()
    
    y_pred = forward(X,V,S)
    mse_loss = nn.MSELoss()
    #print(mse_loss(y_pred,Y))
    #print(reg(V,X,S,.01,.03,.03))
    #+ reg(V,X,S,3,1,3)
    loss = loss_func(y_pred,Y) + reg(V,X,S,3,1,3)
    loss.backward()

    
    with torch.no_grad():
        V -= learning_rate * V.grad

    #need to calculate error
    mes = nn.MSELoss()
    if loss < 0.01:
        print('it converged')
        break
    



tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], dtype=torch.float64)


In [411]:
print(V)

[[0.07514072 0.22781304 0.09419869 ... 0.14812937 0.89535171 0.26911923]
 [0.16373933 0.1186293  0.22807321 ... 0.47775361 0.89335    0.5642668 ]
 [0.44017351 0.38097548 0.07756222 ... 0.32541005 0.32501173 0.93339768]
 ...
 [0.115406   0.200567   0.87017679 ... 0.26915815 0.8351356  0.21022228]
 [0.00708325 0.71062486 0.98077692 ... 0.01685801 0.67188563 0.87969359]
 [0.53139052 0.92698919 0.6167304  ... 0.0345136  0.04086571 0.59086512]]


In [412]:
#Inference stage
outPut = torch.tensor([])

#Label test data
xPrime = torch.tensor(test_vec)
sPrime = torch.tensor(test_sig)
#labels_test is indexes to test_labels_seen
#yTest = torch.tensor(labels_test)

#calculates the outputs
outputs_1 = torch.matmul(torch.matmul(torch.transpose(xPrime,0,1),V),sPrime)
#calculating the predictions
preds_1 = torch.tensor([torch.argmax(output) for output in outputs_1])		

#creating confusion?
cm = confusion_matrix(labels_test, preds_1)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
avg = sum(cm.diagonal())/len(test_labels_unseen)

total = len(preds_1)
correct = 0
for i in range(total):      
    if preds_1[i] == torch.tensor(labels_test[i]):
        correct = correct + 1

print("The top accuracy is:", 100 * correct / (1.0 * total))


TypeError: matmul(): argument 'other' (position 2) must be Tensor, not numpy.ndarray